In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from tasc_core.utils.util_nebuladb import NebulaConnector
from tasc_core.utils.util_shopify_product_parser import ShopifyProductParser
from requests.exceptions import HTTPError  # Add this import

# Load environment variables from .env file
load_dotenv()

username = os.getenv('NEBULA_DB_USERNAME')
password = os.getenv('NEBULA_DB_PASSWORD')

# Get the current working directory
project_path = os.getcwd()

print("Project Path:", project_path)


Project Path: /Users/oladiranadanijo/github_repos/tasc_core/test


In [2]:
nebula = NebulaConnector()
partners_df = nebula.select_df("SELECT * FROM tasc_prod.tasc_xref_partners LIMIT 10")

In [3]:
partners_df.columns


Index(['id', 'asof_dt', 'partner_uid', 'partner_name', 'partner_full_name',
       'partner_desc', 'partner_type', 'partner_primary_url', 'created_at_tms',
       'created_by', 'last_modified_tms', 'last_modified_by', 'source_date',
       'source_datetime'],
      dtype='object')

In [4]:
partners_df

,id,asof_dt,partner_uid,partner_name,partner_full_name,partner_desc,partner_type,partner_primary_url,created_at_tms,created_by,last_modified_tms,last_modified_by,source_date,source_datetime
0,1,2024-11-09,1,Houser of Errors,None,None,VENDOR,https://www.houseoferrors.org/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
1,2,2024-11-09,2,Sage NationÊ,None,None,VENDOR,https://sagenation.uk/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
2,3,2024-11-09,3,Abaga,Abaga Velli,None,VENDOR,https://abagavelli.com/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
3,4,2024-11-09,4,A day without angstÊ,A day without angstÊ,None,VENDOR,https://adwoa.info/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
4,5,2024-11-09,5,Two JeysÊ,None,None,VENDOR,https://twojeys.com/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
5,6,2024-11-09,6,Serge DeNimes,None,None,VENDOR,https://www.sergedenimes.com/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
6,7,2024-11-09,7,CLINTS Inc,None,None,VENDOR,https://clints.co/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
7,8,2024-11-09,8,Corteiz,None,None,VENDOR,https://www.crtz.xyz/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
8,9,2024-11-09,9,Carsico,None,None,VENDOR,https://www.carsick-o.com/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09
9,10,2024-11-09,10,kortsiide,None,None,VENDOR,https://www.kortsiide.com/,None,oadanijo,2024-11-09,oadanijo,2024-11-09,2024-11-09


In [5]:
# Specify the encoding to handle non-UTF-8 characters
# test_df = pd.read_csv('/Users/oladiranadanijo/github_repos/tasc_core/resources/tasc_xref_partners_seed.csv', encoding='ISO-8859-1')

In [6]:
# nebula.insert_df(table_name='tasc_xref_partners', table_schema='tasc_prod', df=test_df)

In [7]:
import requests
import json

def read_json_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    return response.json()

# Example usage
url = 'https://www.houseoferrors.org/products.json'
json_data = read_json_from_url(url)

# Pretty-print JSON data with default indent value of 4
# print(json.dumps(json_data, indent=4))

# Print JSON data without specifying indent (compact format)
# print(json.dumps(json_data))

In [8]:
# Assuming partners_df is already defined and contains a column named 'partner_url'
for index, row in partners_df.iterrows():
    print(row['partner_primary_url'])
    # Create an instance of the parser
    parser = ShopifyProductParser(row['partner_primary_url'] + 'products.json')

    try:
        # Load and parse the JSON data
        parser.load_json()
        parsed_df = parser.to_dataframe()

        # Assuming parsed_df is your DataFrame
        for column in parsed_df.columns:
            parsed_df[column] = parsed_df[column].astype(str).str[:255]
        
        # Convert all columns to object data type
        # parsed_df = parsed_df.astype('object')

        # Display the DataFrame with every field captured
        display(parsed_df)
        print(parsed_df.columns)
        print(parsed_df.dtypes)
        print(len(parsed_df))

        # Print the distinct count of product_title
        distinct_product_titles = parsed_df['handle'].nunique()
        print(f"Distinct count of product_title: {distinct_product_titles}")

        # Insert the DataFrame into the database table
        parsed_df['asof_dt'] = pd.to_datetime('today').date()  # Add asof_dt column
        parsed_df['created_at_tms'] = pd.to_datetime('now')  # Add created_at_tms column
        parsed_df['created_by'] = 'oadanijo'  # Add created_by column
        parsed_df['last_modified_tms'] = pd.to_datetime('now')  # Add last_modified_tms column
        parsed_df['last_modified_by'] = 'oadanijo'  # Add last_modified_by column
        parsed_df['source_date'] = pd.to_datetime('today').date()  # Add source_date column
        parsed_df['source_datetime'] = pd.to_datetime('now')  # Add source_datetime column

        nebula.insert_df(table_name='tasc_products_shopify', table_schema='tasc_prod', df=parsed_df)

    except ValueError as e:
        print(f"Error processing {row['partner_primary_url']}: {e}")
    except HTTPError as e:
        print(f"HTTP error processing {row['partner_primary_url']}: {e}")
    except Exception as e:
        print(f"Unexpected error processing {row['partner_primary_url']}: {e}")
    # break

https://www.houseoferrors.org/


,product_id,product_title,handle,vendor,product_type,tags,published_at,created_at,updated_at,variant_id,...,sku,price,grams,available,requires_shipping,taxable,featured_image,position,image,options
0,8581629051043,KNITTED MOTO TOP,knitted-moto-top,HOUSE OF ERRORS,,,2024-09-01T16:20:43+01:00,2024-08-31T18:38:34+01:00,2024-09-11T21:40:10+01:00,45156926685347,...,None,140.00,1000,False,True,True,None,1,https://cdn.shopify.com/s/files/1/0327/7452/09...,Size
1,8581629051043,KNITTED MOTO TOP,knitted-moto-top,HOUSE OF ERRORS,,,2024-09-01T16:20:43+01:00,2024-08-31T18:38:34+01:00,2024-09-11T21:40:10+01:00,45156926718115,...,None,140.00,1000,False,True,True,None,2,https://cdn.shopify.com/s/files/1/0327/7452/09...,Size
2,8581629051043,KNITTED MOTO TOP,knitted-moto-top,HOUSE OF ERRORS,,,2024-09-01T16:20:43+01:00,2024-08-31T18:38:34+01:00,2024-09-11T21:40:10+01:00,45156926750883,...,None,140.00,1000,False,True,True,None,3,https://cdn.shopify.com/s/files/1/0327/7452/09...,Size
3,8581629051043,KNITTED MOTO TOP,knitted-moto-top,HOUSE OF ERRORS,,,2024-09-01T16:20:43+01:00,2024-08-31T18:38:34+01:00,2024-09-11T21:40:10+01:00,45156926783651,...,None,140.00,1000,False,True,True,None,4,https://cdn.shopify.com/s/files/1/0327/7452/09...,Size
4,8581038997667,KNITTED MOTO JACKET,knitted-moto-jacket,HOUSE OF ERRORS,,,2024-09-01T16:19:30+01:00,2024-08-30T19:51:45+01:00,2024-09-11T21:40:10+01:00,45155570188451,...,HOE-MOTO-JKT-XS,220.00,700,True,True,True,None,1,https://cdn.shopify.com/s/files/1/0327/7452/09...,Size
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,8446829232291,EMBROIDERED TENNIS SHIRT IN BLOSSOM,embroidered-tennis-shirt-blossom,HOUSE OF ERRORS,,,2024-05-26T15:34:43+01:00,2024-05-26T13:37:40+01:00,2024-09-11T21:40:10+01:00,44612208263331,...,HOE-TNS-BLS-S,120.00,0,False,True,True,None,3,https://cdn.shopify.com/s/files/1/0327/7452/09...,Size
151,8446829232291,EMBROIDERED TENNIS SHIRT IN BLOSSOM,embroidered-tennis-shirt-blossom,HOUSE OF ERRORS,,,2024-05-26T15:34:43+01:00,2024-05-26T13:37:40+01:00,2024-09-11T21:40:10+01:00,44612208296099,...,HOE-TNS-BLS-M,120.00,0,False,True,True,None,4,https://cdn.shopify.com/s/files/1/0327/7452/09...,Size
152,8446829232291,EMBROIDERED TENNIS SHIRT IN BLOSSOM,embroidered-tennis-shirt-blossom,HOUSE OF ERRORS,,,2024-05-26T15:34:43+01:00,2024-05-26T13:37:40+01:00,2024-09-11T21:40:10+01:00,44612208328867,...,HOE-TNS-BLS-L,120.00,0,False,True,True,None,5,https://cdn.shopify.com/s/files/1/0327/7452/09...,Size
153,8446829232291,EMBROIDERED TENNIS SHIRT IN BLOSSOM,embroidered-tennis-shirt-blossom,HOUSE OF ERRORS,,,2024-05-26T15:34:43+01:00,2024-05-26T13:37:40+01:00,2024-09-11T21:40:10+01:00,44612208361635,...,HOE-TNS-BLS-XL,120.00,0,False,True,True,None,6,https://cdn.shopify.com/s/files/1/0327/7452/09...,Size


Index(['product_id', 'product_title', 'handle', 'vendor', 'product_type',
       'tags', 'published_at', 'created_at', 'updated_at', 'variant_id',
       'variant_title', 'option_1', 'sku', 'price', 'grams', 'available',
       'requires_shipping', 'taxable', 'featured_image', 'position', 'image',
       'options'],
      dtype='object')
product_id           object
product_title        object
handle               object
vendor               object
product_type         object
tags                 object
published_at         object
created_at           object
updated_at           object
variant_id           object
variant_title        object
option_1             object
sku                  object
price                object
grams                object
available            object
requires_shipping    object
taxable              object
featured_image       object
position             object
image                object
options              object
dtype: object
155
Distinct count of product_

,product_id,product_title,handle,vendor,product_type,tags,published_at,created_at,updated_at,variant_id,...,sku,price,grams,available,requires_shipping,taxable,featured_image,position,image,options
0,9576264630554,Box Pleat Trouser - Brown,box-pleat-trouser-brown,sageNation,Trouser,AW24,2024-09-05T09:31:17+01:00,2024-09-05T09:29:12+01:00,2024-09-11T20:15:52+01:00,49612381290778,...,S011-BLA-S,340.00,0,True,True,True,None,1,https://cdn.shopify.com/s/files/1/0883/1693/64...,Size
1,9576264630554,Box Pleat Trouser - Brown,box-pleat-trouser-brown,sageNation,Trouser,AW24,2024-09-05T09:31:17+01:00,2024-09-05T09:29:12+01:00,2024-09-11T20:15:52+01:00,49612381323546,...,S011-BLA-M,340.00,0,True,True,True,None,2,https://cdn.shopify.com/s/files/1/0883/1693/64...,Size
2,9576264630554,Box Pleat Trouser - Brown,box-pleat-trouser-brown,sageNation,Trouser,AW24,2024-09-05T09:31:17+01:00,2024-09-05T09:29:12+01:00,2024-09-11T20:15:52+01:00,49612381356314,...,S011-BLA-L,340.00,0,True,True,True,None,3,https://cdn.shopify.com/s/files/1/0883/1693/64...,Size
3,9484957614362,Knot Bag - Red Check,s105-red,sageNation,Accessories,AW24,2024-09-05T09:28:39+01:00,2024-06-27T13:52:44+01:00,2024-09-11T20:15:52+01:00,49257488843034,...,S105-RED-OS,350.00,0,True,True,True,None,1,https://cdn.shopify.com/s/files/1/0883/1693/64...,Size
4,9484958761242,Malay Trouser - Black Corduroy,s078-bla,sageNation,Trouser,AW24,2024-09-05T09:27:55+01:00,2024-06-27T13:53:08+01:00,2024-09-11T20:15:52+01:00,49257492021530,...,S078-BLA-S,395.00,0,True,True,True,None,1,https://cdn.shopify.com/s/files/1/0883/1693/64...,Size
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,9484959056154,Welt Trouser Denim - Black Wash,s083-bla,sageNation,Trouser,AW24,2024-06-27T13:53:13+01:00,2024-06-27T13:53:13+01:00,2024-09-11T20:15:52+01:00,49257493004570,...,S083-BLA-M,460.00,0,True,True,True,None,2,https://cdn.shopify.com/s/files/1/0883/1693/64...,Size
91,9484959056154,Welt Trouser Denim - Black Wash,s083-bla,sageNation,Trouser,AW24,2024-06-27T13:53:13+01:00,2024-06-27T13:53:13+01:00,2024-09-11T20:15:52+01:00,49257493037338,...,S083-BLA-L,460.00,0,True,True,True,None,3,https://cdn.shopify.com/s/files/1/0883/1693/64...,Size
92,9484958990618,Petrol Shirt - Green,s082-gre,sageNation,Shirt,AW24,2024-06-27T13:53:12+01:00,2024-06-27T13:53:12+01:00,2024-09-11T20:15:52+01:00,49257492807962,...,S082-GRE-S,390.00,0,True,True,True,None,1,https://cdn.shopify.com/s/files/1/0883/1693/64...,Size
93,9484958990618,Petrol Shirt - Green,s082-gre,sageNation,Shirt,AW24,2024-06-27T13:53:12+01:00,2024-06-27T13:53:12+01:00,2024-09-11T20:15:52+01:00,49257492840730,...,S082-GRE-M,390.00,0,True,True,True,None,2,https://cdn.shopify.com/s/files/1/0883/1693/64...,Size


Index(['product_id', 'product_title', 'handle', 'vendor', 'product_type',
       'tags', 'published_at', 'created_at', 'updated_at', 'variant_id',
       'variant_title', 'option_1', 'sku', 'price', 'grams', 'available',
       'requires_shipping', 'taxable', 'featured_image', 'position', 'image',
       'options'],
      dtype='object')
product_id           object
product_title        object
handle               object
vendor               object
product_type         object
tags                 object
published_at         object
created_at           object
updated_at           object
variant_id           object
variant_title        object
option_1             object
sku                  object
price                object
grams                object
available            object
requires_shipping    object
taxable              object
featured_image       object
position             object
image                object
options              object
dtype: object
95
Distinct count of product_t

...

Index(['product_id', 'product_title', 'handle', 'vendor', 'product_type',
       'tags', 'published_at', 'created_at', 'updated_at', 'variant_id',
       'variant_title', 'option_1', 'sku', 'price', 'grams', 'available',
       'requires_shipping', 'taxable', 'featured_image', 'position', 'image',
       'options'],
      dtype='object')
product_id           object
product_title        object
handle               object
vendor               object
product_type         object
tags                 object
published_at         object
created_at           object
updated_at           object
variant_id           object
variant_title        object
option_1             object
sku                  object
price                object
grams                object
available            object
requires_shipping    object
taxable              object
featured_image       object
position             object
image                object
options              object
dtype: object
76
Distinct count of product_t

,product_id,product_title,handle,vendor,product_type,tags,published_at,created_at,updated_at,variant_id,...,sku,price,grams,available,requires_shipping,taxable,featured_image,position,image,options
0,9535662915743,Leather Clutch Bags,leather-clutch-bag-black-copy,CLINTS Inc,Apparel & Accessories,,2024-09-08T15:59:59+01:00,2024-08-18T13:40:29+01:00,2024-09-11T21:40:17+01:00,46589285466271,...,Yellow,95.00,0,True,True,False,"{'id': 40149788852383, 'product_id': 953566291...",1,https://cdn.shopify.com/s/files/1/0141/6249/34...,Color
1,9535662915743,Leather Clutch Bags,leather-clutch-bag-black-copy,CLINTS Inc,Apparel & Accessories,,2024-09-08T15:59:59+01:00,2024-08-18T13:40:29+01:00,2024-09-11T21:40:17+01:00,46589291954335,...,Green,95.00,0,True,True,False,"{'id': 42886137020575, 'product_id': 953566291...",2,https://cdn.shopify.com/s/files/1/0141/6249/34...,Color
2,9535662915743,Leather Clutch Bags,leather-clutch-bag-black-copy,CLINTS Inc,Apparel & Accessories,,2024-09-08T15:59:59+01:00,2024-08-18T13:40:29+01:00,2024-09-11T21:40:17+01:00,46589291987103,...,Red,95.00,0,True,True,False,"{'id': 40150052962463, 'product_id': 953566291...",3,https://cdn.shopify.com/s/files/1/0141/6249/34...,Color
3,9535662915743,Leather Clutch Bags,leather-clutch-bag-black-copy,CLINTS Inc,Apparel & Accessories,,2024-09-08T15:59:59+01:00,2024-08-18T13:40:29+01:00,2024-09-11T21:40:17+01:00,46589292019871,...,Black,95.00,0,True,True,False,"{'id': 39622161727647, 'product_id': 953566291...",4,https://cdn.shopify.com/s/files/1/0141/6249/34...,Color
4,9551340372127,Phat Denim Jorts,jorts,CLINTS Inc,,,2024-09-04T10:00:37+01:00,2024-09-04T10:00:37+01:00,2024-09-11T21:40:17+01:00,46653819486367,...,BLUE,105.00,0,True,True,False,"{'id': 39622880428191, 'product_id': 955134037...",1,https://cdn.shopify.com/s/files/1/0141/6249/34...,"Color, Size"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,9160858763423,V2 Sweatpants - Triple Black,v2-sweatpants-triple-black,CLINTS Inc,bottoms,"bottoms, clints, joggers, pants, sweatpants, t...",2024-09-03T10:43:58+01:00,2024-02-03T16:16:22+00:00,2024-09-11T21:40:17+01:00,44903813644447,...,None,90.00,0,False,True,True,None,4,https://cdn.shopify.com/s/files/1/0141/6249/34...,Size
182,7681004699807,Footprint Socks,clints-socks,CLINTS Inc,,All products,2024-09-03T10:37:24+01:00,2023-04-09T18:27:49+01:00,2024-09-11T21:40:17+01:00,43064009556127,...,Brown,10.00,0,True,True,True,"{'id': 39628967772319, 'product_id': 768100469...",1,https://cdn.shopify.com/s/files/1/0141/6249/34...,Color
183,7681004699807,Footprint Socks,clints-socks,CLINTS Inc,,All products,2024-09-03T10:37:24+01:00,2023-04-09T18:27:49+01:00,2024-09-11T21:40:17+01:00,43064009588895,...,Green,10.00,0,True,True,True,"{'id': 39628967739551, 'product_id': 768100469...",2,https://cdn.shopify.com/s/files/1/0141/6249/34...,Color
184,7681004699807,Footprint Socks,clints-socks,CLINTS Inc,,All products,2024-09-03T10:37:24+01:00,2023-04-09T18:27:49+01:00,2024-09-11T21:40:17+01:00,43064009621663,...,White/red,10.00,0,True,True,True,"{'id': 39628967837855, 'product_id': 768100469...",3,https://cdn.shopify.com/s/files/1/0141/6249/34...,Color


Index(['product_id', 'product_title', 'handle', 'vendor', 'product_type',
       'tags', 'published_at', 'created_at', 'updated_at', 'variant_id',
       'variant_title', 'option_1', 'sku', 'price', 'grams', 'available',
       'requires_shipping', 'taxable', 'featured_image', 'position', 'image',
       'options'],
      dtype='object')
product_id           object
product_title        object
handle               object
vendor               object
product_type         object
tags                 object
published_at         object
created_at           object
updated_at           object
variant_id           object
variant_title        object
option_1             object
sku                  object
price                object
grams                object
available            object
requires_shipping    object
taxable              object
featured_image       object
position             object
image                object
options              object
dtype: object
186
Distinct count of product_

,product_id,product_title,handle,vendor,product_type,tags,published_at,created_at,updated_at,variant_id,...,sku,price,grams,available,requires_shipping,taxable,featured_image,position,image,options
0,7584924467279,RODMAN TEE | BLACK (COMING SOON),rodman-black,kortsiide,,,2024-09-07T02:21:03+01:00,2024-09-07T02:17:24+01:00,2024-09-11T20:19:26+01:00,42141835952207,...,None,45.00,0,False,True,True,None,1,https://cdn.shopify.com/s/files/1/0559/5604/59...,Size
1,7584924467279,RODMAN TEE | BLACK (COMING SOON),rodman-black,kortsiide,,,2024-09-07T02:21:03+01:00,2024-09-07T02:17:24+01:00,2024-09-11T20:19:26+01:00,42141835984975,...,None,45.00,0,False,True,True,None,2,https://cdn.shopify.com/s/files/1/0559/5604/59...,Size
2,7584924467279,RODMAN TEE | BLACK (COMING SOON),rodman-black,kortsiide,,,2024-09-07T02:21:03+01:00,2024-09-07T02:17:24+01:00,2024-09-11T20:19:26+01:00,42141836017743,...,None,45.00,0,False,True,True,None,3,https://cdn.shopify.com/s/files/1/0559/5604/59...,Size
3,7584924467279,RODMAN TEE | BLACK (COMING SOON),rodman-black,kortsiide,,,2024-09-07T02:21:03+01:00,2024-09-07T02:17:24+01:00,2024-09-11T20:19:26+01:00,42141836050511,...,None,45.00,0,False,True,True,None,4,https://cdn.shopify.com/s/files/1/0559/5604/59...,Size
4,7581762682959,KORTS RUNNA TRACK PANTS | TRIPLE BLACK,korts-runna-track-pants-triple-black,kortsiide,,,2024-09-02T19:06:00+01:00,2024-09-02T17:47:17+01:00,2024-09-11T20:19:26+01:00,42133125333071,...,None,75.00,0,True,True,True,None,2,https://cdn.shopify.com/s/files/1/0559/5604/59...,Size
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,7187765657679,PRE ORDER ONLY - SUPERSTAR SET | DUSK GREY,superstar-set-dusk-grey,kortsiide,,,2023-12-05T20:25:38+00:00,2023-12-05T20:07:56+00:00,2024-09-11T20:19:26+01:00,40975735849039,...,,160.00,0,False,True,True,None,2,https://cdn.shopify.com/s/files/1/0559/5604/59...,Size
138,7187765657679,PRE ORDER ONLY - SUPERSTAR SET | DUSK GREY,superstar-set-dusk-grey,kortsiide,,,2023-12-05T20:25:38+00:00,2023-12-05T20:07:56+00:00,2024-09-11T20:19:26+01:00,40975735881807,...,,160.00,0,False,True,True,None,3,https://cdn.shopify.com/s/files/1/0559/5604/59...,Size
139,7187765657679,PRE ORDER ONLY - SUPERSTAR SET | DUSK GREY,superstar-set-dusk-grey,kortsiide,,,2023-12-05T20:25:38+00:00,2023-12-05T20:07:56+00:00,2024-09-11T20:19:26+01:00,40975735914575,...,,160.00,0,False,True,True,None,4,https://cdn.shopify.com/s/files/1/0559/5604/59...,Size
140,7187765657679,PRE ORDER ONLY - SUPERSTAR SET | DUSK GREY,superstar-set-dusk-grey,kortsiide,,,2023-12-05T20:25:38+00:00,2023-12-05T20:07:56+00:00,2024-09-11T20:19:26+01:00,40975735980111,...,,160.00,0,False,True,True,None,5,https://cdn.shopify.com/s/files/1/0559/5604/59...,Size


Index(['product_id', 'product_title', 'handle', 'vendor', 'product_type',
       'tags', 'published_at', 'created_at', 'updated_at', 'variant_id',
       'variant_title', 'option_1', 'sku', 'price', 'grams', 'available',
       'requires_shipping', 'taxable', 'featured_image', 'position', 'image',
       'options'],
      dtype='object')
product_id           object
product_title        object
handle               object
vendor               object
product_type         object
tags                 object
published_at         object
created_at           object
updated_at           object
variant_id           object
variant_title        object
option_1             object
sku                  object
price                object
grams                object
available            object
requires_shipping    object
taxable              object
featured_image       object
position             object
image                object
options              object
dtype: object
142
Distinct count of product_